In [62]:
# make simple pytorch neural network
import torch
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.optim import Adam
#import resample
from sklearn.utils import resample
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import numpy as np

In [63]:
#load data
data = np.load('data/processed/concatenated_features.npy')

SEED = 1706
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 64

In [64]:
x_majority.shape[0] / x_minority.shape[0]

5.247169200905856

In [161]:
# split into training and test set
x_train, x_test, y_train, y_test = train_test_split(data[:, :-1], data[:, -1], test_size=0.2, random_state=SEED, shuffle=True)

# upsample minority class
x_minority = x_train[y_train == 0]
y_minority = y_train[y_train == 0]
x_majority = x_train[y_train == 1]
y_majority = y_train[y_train == 1]

x_majority_subsample, y_majority_subsample = resample(x_majority, y_majority, replace=False, n_samples=x_minority.shape[0], random_state=SEED)

x_train_balanced = np.concatenate((x_majority_subsample, x_minority))
y_train_balanced = np.concatenate((y_majority_subsample, y_minority))


In [162]:
print(torch.__version__)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')


if torch.cuda.is_available():
    !nvidia-smi

1.10.1+cu113
Using cuda device
Fri Dec 24 00:18:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   21C    P0    55W / 400W |   6368MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:0F:00.0 Off | 

In [163]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(x_train_balanced), 
                       torch.FloatTensor(y_train_balanced))
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(x_test))

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1)

In [164]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.linear_1 = nn.Linear(5595, 10024)
        self.linear_2 = nn.Linear(10024, 1024)
        self.linear_3 = nn.Linear(1024, 256)
        self.output_layer = nn.Linear(256, 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.batch_norm1 = nn.BatchNorm1d(10024)
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.batch_norm3 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.batch_norm1(x)
        x = self.dropout(x)
        x = self.linear_2(x)
        x = self.relu(x)
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = self.linear_3(x)
        x = self.relu(x)
        x = self.batch_norm3(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        
        return x
    

In [165]:
model = NeuralNetwork()
model.to(device)
print(model)

NeuralNetwork(
  (linear_1): Linear(in_features=5595, out_features=10024, bias=True)
  (linear_2): Linear(in_features=10024, out_features=1024, bias=True)
  (linear_3): Linear(in_features=1024, out_features=256, bias=True)
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (batch_norm1): BatchNorm1d(10024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [166]:
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

In [167]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [108]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in tqdm(train_dataloader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_dataloader):.5f} | Acc: {epoch_acc/len(train_dataloader):.3f}')

100%|██████████| 194/194 [00:01<00:00, 133.15it/s]


Epoch 001: | Loss: 0.44791 | Acc: 79.263


100%|██████████| 194/194 [00:01<00:00, 132.68it/s]


Epoch 002: | Loss: 0.36266 | Acc: 83.835


100%|██████████| 194/194 [00:01<00:00, 123.84it/s]


...

Epoch 048: | Loss: 0.05712 | Acc: 97.753


100%|██████████| 194/194 [00:01<00:00, 134.88it/s]


Epoch 049: | Loss: 0.05289 | Acc: 97.845


100%|██████████| 194/194 [00:01<00:00, 124.46it/s]

Epoch 050: | Loss: 0.05193 | Acc: 98.041


In [109]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_dataloader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [110]:
confusion_matrix(y_test, y_pred_list)

array([[1506,   34],
       [ 503, 7612]])

In [111]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.75      0.98      0.85      1540
         1.0       1.00      0.94      0.97      8115

    accuracy                           0.94      9655
   macro avg       0.87      0.96      0.91      9655
weighted avg       0.96      0.94      0.95      9655



## Shuffle a column at a time

In [173]:
# single-column shuffle iterator
def sc(x):
    p = X.shape[1]
    for i in range(p):
        hold = np.array(x[:,i])
        np.random.shuffle(x[:,i])
        yield x
        x[:,i] = hold


# copy x_train_balanced
x_train_shuffle = x_train_balanced.copy()

with tqdm(total=x_train_shuffle.shape[1]) as pbar:
    for i, shuffled_data in enumerate(sc(x_train_shuffle)):
        train_data = TrainData(torch.FloatTensor(shuffled_data), torch.FloatTensor(y_train_balanced))
        train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

        model.train()
        for e in range(1, EPOCHS+1):
            epoch_loss = 0
            epoch_acc = 0
            for X_batch, y_batch in train_dataloader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()

                y_pred = model(X_batch)

                loss = criterion(y_pred, y_batch.unsqueeze(1))
                acc = binary_acc(y_pred, y_batch.unsqueeze(1))

                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                epoch_acc += acc.item()
            tqdm.write(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_dataloader):.5f} | Acc: {epoch_acc/len(train_dataloader):.3f}')
                
        pbar.update(1)



  0%|          | 0/5595 [00:01<?, ?it/s]

Epoch 001: | Loss: 0.04220 | Acc: 98.418


  0%|          | 0/5595 [00:02<?, ?it/s]

Epoch 002: | Loss: 0.04663 | Acc: 98.088


  0%|          | 0/5595 [00:04<?, ?it/s]

...

Epoch 042: | Loss: 0.03503 | Acc: 98.758


  0%|          | 0/5595 [01:02<?, ?it/s]

Epoch 043: | Loss: 0.03135 | Acc: 98.711


  0%|          | 0/5595 [01:03<?, ?it/s]


KeyboardInterrupt: 